# GSEG data validation

This notebook contains code that can be used to validate data created during GSEG3. It reads in the APT-derived xml and pointing files and constructs a dictionary of expected data properties. It then compares these properties to the information contained in the headers of the actual data to look for inconsistencies.

**Mirage and pysiaf are dependencies. Be sure they are installed in your environment.**

In [2]:
from astropy.io import fits
from astropy.table import Table
from collections import OrderedDict
from glob import glob
from mirage.yaml import yaml_generator
from mirage.apt import apt_inputs
from mirage.seed_image.catalog_seed_image import Catalog_seed
from mirage.utils.siaf_interface import sci_subarray_corners
from mirage.utils.utils import calc_frame_time
from mirage.yaml.generate_observationlist import get_observation_dict
import numpy as np
import os
import pkg_resources
import pysiaf

## User Inputs

In [3]:
# The observations to validate (use 3 characters); if you want to validate all
# observations in a proposal, just set observations = []
observations = ['191','192','193','194','195','196','197','198', '199', '200', '201']

In [4]:
# Header keywords to check against APT-derived dictionary
# It is assumed that these header keywords exist in both the uncal and rate images
KEYWORDS = ['SUBARRAY', 'DETECTOR', 'NINTS', 'NGROUPS', 'NAXIS', 'EFFEXPTM',
            'LONGFILTER', 'LONGPUPIL', 'SHORTFILTER', 'SHORTPUPIL', 'READPATT',
            'OBSLABEL', 'EXP_TYPE', 'TITLE', 'OBSERVTN', 'TEMPLATE',
            'EXPRIPAR', 'SUBSTRT1', 'SUBSTRT2', 'SUBSIZE1', 'SUBSIZE2',
            'FASTAXIS', 'SLOWAXIS', 'PATTTYPE']

In [5]:
# Corresponding keywords in the APT-derived dictionary
# These must correspond one-to-one with KEYWORDS above
TABLE_KEYWORDS = ['Subarray', None, 'Integrations', 'Groups', None, None,
                  'LongFilter', 'LongPupil', 'ShortFilter', 'ShortPupil', 'ReadoutPattern',
                  'ObservationName', 'Mode', 'Title', 'ObservationID', 'APTTemplate',
                  'ParallelInstrument', None, None, None, None,
                  None, None, 'PrimaryDitherType']

In [6]:
# Dictionary of header keywords and their expected values in the 
# PRIMARY and SCI extensions of the UNCAL, DARK, and RATE files.
# These are useful when you know exactly what these keywords should be
# for every file (e.g. NAXIS should always be 2 in the RATE image SCI header).
# These will be checked in addition to the KEYWORDS checks above
UNCAL_PRIMARY_KEYWORDS = {}
UNCAL_SCI_KEYWORDS = {'BITPIX':16, 'NAXIS':4, 'BUNIT':'DN'}
DARK_PRIMARY_KEYWORDS = {}
DARK_SCI_KEYWORDS = {'NAXIS':4, 'BUNIT':'DN'}
RATE_PRIMARY_KEYWORDS = {}
RATE_SCI_KEYWORDS = {'BITPIX':-32, 'NAXIS':2, 'BUNIT':'DN/s'}

In [7]:
# The PRIMARY and SCI header keywords to store for each file in the output summary table
SUMMARY_TABLE_PRIMARY = ['FILENAME', 'DETECTOR', 'FILTER', 'PUPIL', 'EXP_TYPE', 'READPATT', 'NINTS', 'NGROUPS',
                         'NFRAMES', 'GROUPGAP', 'SUBARRAY', 'SUBSTRT1', 'SUBSTRT2', 'SUBSIZE1', 'SUBSIZE2', 
                         'APERNAME']
SUMMARY_TABLE_SCI = ['BITPIX', 'NAXIS', 'NAXIS1', 'NAXIS2', 'NAXIS3', 'NAXIS4', 'BUNIT']

## Define some constants

In [8]:
INTEGER_KEYWORDS = ['Integrations', 'Groups']
FLOAT_KEYWORDS = ['EFFEXPTM']
FILTER_KEYWORDS = ['LONGFILTER', 'LONGPUPIL', 'SHORTFILTER', 'SHORTPUPIL']

In [9]:
# For FASTAXIS and SLOWAXIS
HORIZONTAL_FLIP = ['NRCA1', 'NRCA3', 'NRCALONG', 'NRCB2', 'NRCB4']
VERTICAL_FLIP = ['NRCA2', 'NRCA4', 'NRCB1', 'NRCB3', 'NRCBLONG']

In [10]:
# Expected detectors used for each module_subarray combo (these are not always perfect)
DETECTOR_DICT = {'ALL_FULL':['NRCA1', 'NRCA2', 'NRCA3', 'NRCA4', 'NRCALONG',
                             'NRCB1', 'NRCB2', 'NRCB3', 'NRCB4', 'NRCBLONG'],
                 'A_FULL':['NRCA1', 'NRCA2', 'NRCA3', 'NRCA4', 'NRCALONG'],
                 'A_SUBGRISM256':['NRCA1', 'NRCA3','NRCALONG'],
                 'A_SUBGRISM128':['NRCA1', 'NRCA3','NRCALONG'],
                 'A_SUBGRISM64':['NRCA1', 'NRCA3','NRCALONG'],
                 'B_FULL':['NRCB1', 'NRCB2', 'NRCB3', 'NRCB4', 'NRCBLONG'],
                 'B_SUB640':['NRCB1', 'NRCB2', 'NRCB3', 'NRCB4', 'NRCBLONG'],
                 'B_SUB320':['NRCB1', 'NRCB2', 'NRCB3', 'NRCB4', 'NRCBLONG'],
                 'B_SUB160':['NRCB1', 'NRCB2', 'NRCB3', 'NRCB4', 'NRCBLONG'],
                 'B_SUB400P':['NRCB1', 'NRCBLONG'],
                 'B_SUB160P':['NRCB1', 'NRCBLONG'],
                 'B_SUB64P':['NRCB1', 'NRCBLONG']}

## Define functions

In [11]:
def add_entry(filename, summary_dict):
    """Adds an entry for filename to the output summary dict
    """
    for key in SUMMARY_TABLE_PRIMARY:
        try:
            val = str(fits.getheader(filename, 'PRIMARY')[key])
        except KeyError:
            val = ''
        summary_dict[key].append(val)
    
    for key in SUMMARY_TABLE_SCI:
        try:
            val = str(fits.getheader(filename, 'SCI')[key])
        except KeyError:
            val = ''
        summary_dict[key].append(val)
    
    return summary_dict

In [12]:
def adjust_exptype(value):
    """Modify the exposure type as listed in the exposure table
    to match one of the strings as used in the fits files.
    e.g. 'imaging' becomes 'NRC_IMAGE'
    Remember that currently, Mirage only knows imaging and wfss
    """
    if value == 'imaging':
        return 'NRC_IMAGE'
    elif value == 'wfss':
        return 'NRC_GRISM'

In [13]:
def calculate_total_files(exp_dict, index):
    """Calculate the total number of files expected for an
    observation based on the number of dithers and the 
    module/subarray combination.
    """
    module = exp_dict['Module'][index].upper()
    subarray = exp_dict['Subarray'][index].upper()
    subarray = subarray.replace('DHSPILA', '').replace('DHSPILB', '')
    number_of_dithers = exp_dict['number_of_dithers'][index]
    combo = '{}_{}'.format(module, subarray)
    
    # Find expected number of detectors based on module/subarray combo
    try:
        expected_detectors = DETECTOR_DICT[combo]
        n_detectors = len(expected_detectors)
    except KeyError:
        print('Warning: module/subbary={} not an expected '
              'combination; assuming all 10 detectors used.'.format(combo))
        n_detectors = 10
        expected_detectors = DETECTOR_DICT['ALL_FULL']
    
    # Find total expected number of files/detectors based on detectors/dithers
    total = number_of_dithers * n_detectors
    expected_detectors = expected_detectors * number_of_dithers
    
    return total, expected_detectors

In [14]:
def check_apername(sub, aperture):
    """Check that APT subarray is consistent with header APERNAME"""
    n = [s for s in sub if s.isdigit()]
    n = [''.join(n[:])][0]  # number from subarray
    nn = [s for s in aperture.split('_')[1] if s.isdigit()]  # split[1] to get rid of ends of e.g. NRCA5_GRISM256_F277W
    nn = [''.join(nn[:])][0]  # number from header aperture
    if ((n != nn) |
        (('GRISM' in sub) & ('GRISM' not in aperture)) |
        (('GRISM' in aperture) & ('GRISM' not in sub)) |
        ((n+'P' in sub) & (n+'P' not in aperture)) |
        ((n+'P' in aperture) & (n+'P' not in sub)) |
        (('FULL' in sub) & ('FULL' not in aperture)) |
        (('FULL' in aperture) & ('FULL' not in sub))):
        print('WARNING: Mismatch between APT Subarray ({}) and APERNAME ({})'.format(sub, aperture)) 

In [15]:
def check_detector_exposures(matching_uncal_files, matching_rate_files, expected_detectors):
    """Ensures both an uncal and rate file exists for each expected detector.
    """
    expected_detectors = np.array(expected_detectors)
    for detector in expected_detectors:
        n_dets = len(expected_detectors[expected_detectors==detector])
        n_files = 0
        for f in matching_uncal_files:
            if detector.lower() in f:
                n_files += 1
        if n_files != n_dets:
            print('WARNING: Expected {} uncal files for detector {}, but found {}'.format(n_dets, detector, n_files))
        n_files = 0
        for f in matching_rate_files:
            if detector.lower() in f:
                n_files += 1
        if n_files != n_dets:
            print('WARNING: Expected {} rate files for detector {}, but found {}'.format(n_dets, detector, n_files))

In [16]:
def equalize_file_lists(uncal, rate):
    """Given lists of uncal and rate files corresponding to a single
    observation, adjust the lists to be the same length, adding in
    None for any files that are missing in a given list
    """
    udict = {}
    rdict = {}
    expanded_rate = []
    expanded_uncal = []

    # Loop through uncal files and look for matching rate files
    for ufile in uncal:
        dirname, filename = os.path.split(ufile)
        base = filename.strip('_uncal.fits')
        fullbase = os.path.join(dirname, base)
        found = False
        for rfile in rate:
            if fullbase in rfile:
                found = True
                break
        udict[fullbase] = found

    # Loop through rate files and look for matching uncal files
    for rfile in rate:
        dirname, filename = os.path.split(rfile)
        base = filename.strip('_rate.fits')
        fullbase = os.path.join(dirname, base)
        found = False
        for ufile in uncal:
            if fullbase in ufile:
                found = True
                break
        rdict[fullbase] = found

    # Fill in missing files, in either uncal or rate lists,
    # with None
    for ukey in udict:
        expanded_uncal.append(ukey + '_uncal.fits')
        if udict[ukey]:
            expanded_rate.append(ukey + '_rate.fits')
        else:
            expanded_rate.append(None)
    for rkey in rdict:
        if not rdict[rkey]:
            expanded_rate.append(rkey + '_rate.fits')
            expanded_uncal.append(None)
    return expanded_uncal, expanded_rate

In [17]:
def find_fastaxis(detector):
    """Identify the values of FASTAXIS and SLOWAXIS based on the detector
    name
    """
    if detector in HORIZONTAL_FLIP:
        fast = -1
        slow = 2
    elif detector in VERTICAL_FLIP:
        fast = 1
        slow = -2
    return fast, slow

In [18]:
def find_rate_files(gseg_uncal_files):
    """Returns a list of rate files that correspond to the 
    input uncal files. These may be either rate.fits files 
    or dark.fits files."""
    
    gseg_rate_files = []
    for f in gseg_uncal_files:
        rate_file = f.replace('uncal.fits','rate.fits')
        dark_file = f.replace('uncal.fits','dark.fits')
        if os.path.isfile(rate_file):
            gseg_rate_files.append(rate_file)
        elif os.path.isfile(dark_file):
            gseg_rate_files.append(dark_file)
        else:
            print('Warning: No corresponding rate file for {}'.format(f))
    
    return gseg_rate_files

In [19]:
def get_data(filename):
    """Read in the given fits file and return the data and header
    """
    with fits.open(filename) as h:
        signals = h['SCI'].data
        header0 = h[0].header
        header1 = h[1].header
    return signals, header0, header1

In [20]:
def get_expected_shape(sub):
    """Returns the expected shape of the science data
    based on the input APT subarray.
    """
    siaf = pysiaf.Siaf('NIRCam')
    subarray = sub.replace('SUB', '').replace('DHSPILA', '').replace('DHSPILB', '')
    
    if 'FULL' in subarray:
        expected_shape = (2048, 2048)
    else:
        # needed to be careful here to remove cases where e.g. SUB64 was in SUB640
        similar_aps = [aper for aper in siaf.apernames if subarray in aper and subarray+'0' not in aper]
        if len(similar_aps) == 0:
            print('WARNING: Cannot find expected shape for subarray {}'.format(sub))
            expected_shape = (-99, -99)
        else:
            # just use first entry to get expected shape since they should all be the same
            similar_ap = similar_aps[0]
            expected_shape = (siaf[similar_ap].YSciSize, siaf[similar_ap].XSciSize)
    
    return expected_shape

In [21]:
def header_keywords(head):
    """Extract values for the desired keywords from the given header
    """
    file_info = {}
    for keyword in KEYWORDS:
        try:
            info = head[keyword]
        except KeyError:
            if 'FILTER' in keyword:
                info = head['FILTER']
            elif 'PUPIL' in keyword:
                info = head['PUPIL']
            else:
                info = None

        file_info[keyword] = info
    return file_info

In [22]:
def table_info(values, index):
    """Extract information from the exposure table that matches the
    header keyword values in KEYWORDS
    """
    values_dict = {}
    for table_keyword, file_keyword in zip(TABLE_KEYWORDS, KEYWORDS):
        if table_keyword is not None:
            if table_keyword in INTEGER_KEYWORDS:
                value = int(values[table_keyword][index])
            else:
                value = values[table_keyword][index]
            values_dict[file_keyword] = value
        else:
            values_dict[file_keyword] = None
    return values_dict

In [23]:
def verify_dimensions(filename, file_type, expected_shape):
    """Verify the header and data dimensions for each extension.
    """
    header = fits.getheader(filename, 'PRIMARY')
    if file_type == 'UNCAL':
        extensions = ['SCI']
        primary_header_shape = (header['NINTS'], header['NGROUPS'], header['SUBSIZE2'], header['SUBSIZE1'])
        for ext in extensions:
            try:
                header = fits.getheader(filename, ext)
                data_shape = fits.getdata(filename, ext).shape
                naxis_shape = (header['NAXIS4'], header['NAXIS3'], header['NAXIS2'], header['NAXIS1'])
                if ((primary_header_shape != data_shape) | (primary_header_shape != naxis_shape) | 
                    (primary_header_shape[-2:] != expected_shape)):
                    print('WARNING: Data dimensions incorrect')
                    print('Expected image shape: {}'.format(expected_shape))
                    print('PRIMARY header shape: {}'.format(primary_header_shape))
                    print('{} header shape: {}'.format(ext, naxis_shape))
                    print('{} data shape: {}'.format(ext, data_shape)) 
            except KeyError:
                print('Cannot verify shape for {} extension'.format(ext))
    elif file_type == 'DARK':
        extensions = 'SCI PIXELDQ GROUPDQ ERR'.split()
        primary_header_shape = (header['NINTS'], header['NGROUPS'], header['SUBSIZE2'], header['SUBSIZE1'])
        for ext in extensions:
            try:
                header = fits.getheader(filename, ext)
                data_shape = fits.getdata(filename, ext).shape
                # PIXELDQ extension is only 2D and doesnt have NAXIS3/4, 
                # so just make it 4D by appending expected results
                if ext == 'PIXELDQ':
                    data_shape = (primary_header_shape[0], primary_header_shape[1], data_shape[0], data_shape[1])
                    naxis_shape = (primary_header_shape[0], primary_header_shape[1], header['NAXIS2'], header['NAXIS1'])
                else:
                    naxis_shape = (header['NAXIS4'], header['NAXIS3'], header['NAXIS2'], header['NAXIS1'])
                if ((primary_header_shape != data_shape) | (primary_header_shape != naxis_shape) | 
                    (primary_header_shape[-2:] != expected_shape)):
                    print('WARNING: Data dimensions incorrect')
                    print('Expected image shape: {}'.format(expected_shape))
                    print('PRIMARY header shape: {}'.format(primary_header_shape))
                    print('{} header shape: {}'.format(ext, naxis_shape))
                    print('{} data shape: {}'.format(ext, data_shape))
            except KeyError:
                print('Cannot verify shape for {} extension'.format(ext))
    elif file_type == 'RATE':
        extensions = 'SCI ERR DQ VAR_POISSON VAR_RNOISE'.split()
        primary_header_shape = (header['SUBSIZE2'], header['SUBSIZE1'])
        for ext in extensions:
            try:
                header = fits.getheader(filename, ext)
                data_shape = fits.getdata(filename, ext).shape
                naxis_shape = (header['NAXIS2'], header['NAXIS1'])
                if ((primary_header_shape != data_shape) | (primary_header_shape != naxis_shape) | 
                    (primary_header_shape != expected_shape)):
                    print('WARNING: Data dimensions incorrect')
                    print('Expected image shape: {}'.format(expected_shape))
                    print('PRIMARY header shape: {}'.format(primary_header_shape))
                    print('{} header shape: {}'.format(ext, naxis_shape))
                    print('{} data shape: {}'.format(ext, data_shape))
            except KeyError:
                print('Cannot verify shape for {} extension'.format(ext))
    else:
        print('File type {} not supported for dimension checks'.format(file_type))

In [24]:
def verify_extensions(filename, file_type):
    """Verify that the expected extensions exist
    """
    if file_type == 'UNCAL':
        extensions = 'PRIMARY SCI GROUP INT_TIMES ASDF'.split()
    elif file_type == 'DARK':
        extensions = 'PRIMARY SCI PIXELDQ GROUPDQ ERR GROUP INT_TIMES ASDF'.split()
    elif file_type == 'RATE':
        extensions = 'PRIMARY SCI ERR DQ VAR_POISSON VAR_RNOISE ASDF'.split()
    else:
        print('File type {} not supported for ext verification'.format(file_type))
        
    for ext in extensions:
        try:
            header = fits.getheader(filename, ext)
        except KeyError:
            print('WARNING: {} extension does not exist'.format(ext))

## The main function

In [25]:
def validate(xml_file, output_dir, gseg_uncal_files):
    """MAIN FUNCTION"""
    
    read_pattern_def_file = os.path.join(pkg_resources.resource_filename('mirage', ''), 
                                         'config', 'nircam_read_pattern_definitions.list')
    
    # Make an empty dictionary to store output summary table info
    summary_dict = OrderedDict()
    cols = SUMMARY_TABLE_PRIMARY + SUMMARY_TABLE_SCI
    for col in cols:
        summary_dict[col] = []
    
    # Find the corresponding rate files for each uncal file
    gseg_rate_files = find_rate_files(gseg_uncal_files)
    
    # Create apt-derived dictionary
    pointing_file = xml_file.replace('.xml', '.pointing')
    catalogs = {'nircam': {'sw': 'nothing.cat', 'lw': 'nothing.cat'}}
    observation_list_file = os.path.join(output_dir, 'observation_list.yaml')
    apt_xml_dict = get_observation_dict(xml_file, observation_list_file, catalogs,
                                        verbose=True)
    
    # Find observations to validate, either all in proposal or user-specified
    observation_list = set(apt_xml_dict['ObservationID'])
    if len(observations) != 0:
        str_obs_list = observations
    else:
        int_obs = sorted([int(o) for o in observation_list])
        str_obs_list = [str(o).zfill(3) for o in int_obs]
    
    for observation_to_check in str_obs_list:
        print('')
        print('')
        print('OBSERVATION: {}'.format(observation_to_check))
        print('')
        
        good = np.where(np.array(apt_xml_dict['ObservationID']) == observation_to_check)
        module = apt_xml_dict['Module'][good[0][0]].upper()
        
        try:
            total_expected_files, expected_detectors = calculate_total_files(apt_xml_dict, good[0][0])
            print('Total number of expected files: {}'.format(total_expected_files))
            print('Expected detectors used: {}'.format(expected_detectors))
        except IndexError:
            print("No files found.")
            continue

        # The complication here is that the table created by Mirage does not have a filename
        # attached to each entry. So we need a way to connect an actual filename
        # to each entry
        subdir_start = 'jw' + apt_xml_dict['ProposalID'][good[0][0]] + observation_to_check.zfill(3)
        matching_uncal_files = sorted([filename for filename in gseg_uncal_files if subdir_start in filename])
        matching_rate_files = sorted([filename for filename in gseg_rate_files if subdir_start in filename])
        print('Found uncal files:')
        for i in range(len(matching_uncal_files)):
            print(matching_uncal_files[i])
        print('')
        print('Found rate files:')
        for i in range(len(matching_rate_files)):
            print(matching_rate_files[i])
        print('')
        
        # Check for any missing files and that a file exists for each expected detector used
        check_detector_exposures(matching_uncal_files, matching_rate_files, expected_detectors)
        
        # Deal with the case of matching_uncal_files and matching_rate_files having
        # different lengths here. In order to loop over them they must have the same length
        if len(matching_uncal_files) != len(matching_rate_files):
            (matching_uncal_files, matching_rate_files) = equalize_file_lists(matching_uncal_files, matching_rate_files)
            print('Equalized file lists (should have a 1:1 correspondence):')
            for idx in range(len(matching_uncal_files)):
                print(matching_uncal_files[idx], matching_rate_files[idx])

        # Create siaf instance for later calculations
        siaf = pysiaf.Siaf('NIRCam')

        for file_pair in zip(matching_uncal_files, matching_rate_files):
            for f in file_pair: 
                # Only validate files that exist
                good_file = f != None
                if good_file:
                    if not os.path.isfile(f):
                        print('WARNING: File does not exist: {}'.format(f))
                        good_file = False
                
                if good_file:
                    print("Checking {}".format(os.path.split(f)[1]))
                    print('----------------------------------------------------')
                    file_type = f.split('.fits')[0].split('_')[-1].upper()
                else:
                    continue
                
                # Verify that all expected extensions exist for this file and add
                # file info to the output summary table
                verify_extensions(f, file_type)
                summary_dict = add_entry(f, summary_dict)
                
                # Get info from header to be compared
                data, header, sci_header = get_data(f)
                header_vals = header_keywords(header)

                # Get matching data from the APT exposure table
                table_vals = table_info(apt_xml_dict, good[0][0])
                
                # Verify that the header and data dimensions are correct in each extension
                expected_shape = get_expected_shape(table_vals['SUBARRAY'])
                verify_dimensions(f, file_type, expected_shape)
                
                # Check detector/aperture
                detector_from_filename = f.split('_')[-2].upper()
                header_detector = header['DETECTOR']
                aperture = header['APERNAME']  # could also try APERNAME, PPS_APER
                if 'LONG' in header_detector:
                    header_detector = header_detector.replace('LONG', '5')
                if header_detector not in aperture:
                    print(("WARNING: Detector name and aperture name in file header appear to be incompatible: {}, {}"
                          .format(header['DETECTOR'], aperture)))
                    print("Detector listed in filename: {}".format(detector_from_filename))
                    print('If the aperture is incorrect then the calculated subarray '
                          'location from pysiaf will also be incorrect.')
                check_apername(table_vals['SUBARRAY'], aperture)  # make sure APT subarray is consistent with header APERNAME
                data_shape = data.shape
                
                # Compare NFRAME, GROUPGAP from header with expected values based on READPATT
                m = Catalog_seed()
                params = {'Readout': {'readpatt': table_vals['READPATT']},
                          'Reffiles': {'readpattdefs': read_pattern_def_file}}
                m.params = params
                m.read_pattern_check()
                nframes = m.params['Readout']['nframe']
                groupgap = m.params['Readout']['nskip']
                if nframes != header['NFRAMES']:
                    print('WARNING: NFRAME mismatch between header ({}) and expected value ({}).'.format(
                          nframes, header['NFRAMES']))
                if groupgap != header['GROUPGAP']:
                    print('WARNING: GROUPGAP mismatch between header ({}) and expected value ({}).'.format(
                          groupgap, header['GROUPGAP']))

                # Make some adjustments to the exposure table info

                # Calucate the exposure time
                if 'FULL' in table_vals['SUBARRAY']:
                    num_amps = 4
                else:  # assume all grism and sub use 1 amp outputs
                    num_amps = 1
                frametime = calc_frame_time('NIRCam', aperture, data_shape[-1], data_shape[-2], num_amps)
                table_vals['EFFEXPTM'] = frametime * (table_vals['NGROUPS'] * (nframes+groupgap) - groupgap)

                # NAXIS
                table_vals['NAXIS'] = len(data.shape)
                header_vals['NAXIS'] = sci_header['NAXIS']

                # Use pysiaf to calculate subarray locations
                try:
                    xc, yc = sci_subarray_corners('NIRCam', aperture, siaf=siaf)
                    table_vals['SUBSTRT1'] = xc[0] + 1
                    table_vals['SUBSTRT2'] = yc[0] + 1
                    table_vals['SUBSIZE1'] = siaf[aperture].XSciSize
                    table_vals['SUBSIZE2'] = siaf[aperture].YSciSize
                except KeyError:
                    print("ERROR: Aperture {} is not a valid aperture in pysiaf.".format(aperture))
                    xc = [-2, -2]
                    yc = [-2, -2]
                    table_vals['SUBSTRT1'] = xc[0] + 1
                    table_vals['SUBSTRT2'] = yc[0] + 1
                    table_vals['SUBSIZE1'] = 9999
                    table_vals['SUBSIZE2'] = 9999

                # Create FASTAXIS and SLOWAXIS values based on the detector name
                fast, slow = find_fastaxis(header_vals['DETECTOR'])
                table_vals['FASTAXIS'] = fast
                table_vals['SLOWAXIS'] = slow

                # Remove whitespace from observing template in file
                header_vals['TEMPLATE'] = header_vals['TEMPLATE'].replace(' ', '').lower()
                table_vals['TEMPLATE'] = table_vals['TEMPLATE'].lower()

                # Adjust prime/parallel boolean from table to be a string
                if not table_vals['EXPRIPAR']:
                    table_vals['EXPRIPAR'] = 'PRIME'
                else:
                    table_vals['EXPRIPAR'] = 'PARALLEL'

                # Change exposure type from table to match up with
                # types of strings in the file
                table_vals['EXP_TYPE'] = adjust_exptype(table_vals['EXP_TYPE'])

                # Set the DETECTOR field to be identical. This info is not in the
                # exposure table, so we can't actually check it
                table_vals['DETECTOR'] = header_vals['DETECTOR']

                # Now compare the data in the dictionary from the file versus that
                # from the exposure table created from the APT file
                err = False
                for key in header_vals:
                    if header_vals[key] != table_vals[key]:
                        if key not in FLOAT_KEYWORDS and key not in FILTER_KEYWORDS:
                            err = True
                            print('MISMATCH: {}, in exp table: {}, in file: {}'.format(key, table_vals[key], header_vals[key]))
                        elif key in FLOAT_KEYWORDS:
                            if not np.isclose(header_vals[key], table_vals[key], rtol=0.01, atol=0.):
                                err = True
                                print('MISMATCH: {}, in exp table: {}, in file: {}'.format(key, table_vals[key], header_vals[key]))

                        if key in ['LONGFILTER', 'LONGPUPIL'] and 'LONG' in header_vals['DETECTOR']:
                            err = True
                            print('MISMATCH: {}, in exp table: {}, in file: {}'.format(key, table_vals[key], header_vals[key]))
                        if key in ['SHORTFILTER', 'SHORTPUPIL'] and 'LONG' not in header_vals['DETECTOR']:
                            err = True
                            print('MISMATCH: {}, in exp table: {}, in file: {}'.format(key, table_vals[key], header_vals[key]))
                
                # Perform direct comparison between header keywords and their expected values
                if file_type == 'UNCAL':
                    for key in UNCAL_PRIMARY_KEYWORDS:
                        if UNCAL_PRIMARY_KEYWORDS[key] != header[key]:
                            err = True
                            print('MISMATCH: {}, expected: {}, in file: {}'.format(
                                  key, UNCAL_PRIMARY_KEYWORDS[key], header[key]))
                    for key in UNCAL_SCI_KEYWORDS:
                        if UNCAL_SCI_KEYWORDS[key] != sci_header[key]:
                            err = True
                            print('MISMATCH: {}, expected: {}, in file: {}'.format(
                                  key, UNCAL_SCI_KEYWORDS[key], sci_header[key]))
                elif file_type == 'DARK':
                    for key in DARK_PRIMARY_KEYWORDS:
                        if DARK_PRIMARY_KEYWORDS[key] != header[key]:
                            err = True
                            print('MISMATCH: {}, expected: {}, in file: {}'.format(
                                  key, DARK_PRIMARY_KEYWORDS[key], header[key]))
                    for key in DARK_SCI_KEYWORDS:
                        if DARK_SCI_KEYWORDS[key] != sci_header[key]:
                            err = True
                            print('MISMATCH: {}, expected: {}, in file: {}'.format(
                                  key, DARK_SCI_KEYWORDS[key], sci_header[key]))
                elif file_type == 'RATE':
                    for key in RATE_PRIMARY_KEYWORDS:
                        if RATE_PRIMARY_KEYWORDS[key] != header[key]:
                            err = True
                            print('MISMATCH: {}, expected: {}, in file: {}'.format(
                                  key, RATE_PRIMARY_KEYWORDS[key], header[key]))
                    for key in RATE_SCI_KEYWORDS:
                        if RATE_SCI_KEYWORDS[key] != sci_header[key]:
                            err = True
                            print('MISMATCH: {}, expected: {}, in file: {}'.format(
                                  key, RATE_SCI_KEYWORDS[key], sci_header[key]))
                else:
                    print('No direct header checks performed for {} file type.'.format(file_type))

                if not err:
                    print('No inconsistencies. File header info correct.')
                    
                print('')

            print('')
            print('')
    
    # Output the summary table
    summary_table = Table(summary_dict)
    summary_table.write(os.path.join(output_dir, 'summary_table.txt'), format='ascii.fixed_width_two_line')
    

## Run the tool

In [16]:
xml_file = '/path/to/proposal/xml/file/00617.xml'
output_dir = '/location/to/place/outputs/'
gseg_uncal_files = glob('/path/to/gseg/files/*uncal.fits')

In [ ]:
validate(xml_file, output_dir, gseg_uncal_files)

In [36]:
xml_file = '/Volumes/LaCie/gseg_validation_tests/APT_00617_full/617.xml'
output_dir = '/Volumes/LaCie/gseg_validation_tests/gseg3_july2020_RunForTheRecord/outputs/'
gseg_uncal_files = glob('/Volumes/LaCie/gseg_validation_tests/gseg3_july2020_RunForTheRecord/JWST/*/*uncal.fits')
validate(xml_file, output_dir, gseg_uncal_files)

target_info:
{'ALF-CMA-R': ('06:35:19.5590', '-66:49:7.12')}
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Observation `001` labelled `NIRCam EO-1 GSEG3E1-OTB-20190122 ModB-SUB640` uses template `NircamEngineeringImaging`
APTObservationParams Dictionary holds 0 entries before reading template
Primary dither element PrimaryDithers not found, use default primary dithers value (1).
Number of dithers: 1 primary * 1 subpixel = 1
Dictionary read from template has 1 entries.
Found 1 tile(s) for observation 001 NIRCam EO-1 GSEG3E1-OTB-20190122 ModB-SUB640
Found 1 visits with numbers: [1]
APTObservationParams Dictionary holds 1 entries after reading template (+1 entries)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Observation `002` labelled `NIRCam EO-2 GSEG3E1-OTB-20190122 ModB-SUB320` uses template `NircamEngineeringImaging`
APTObservationParams Dictionary holds 1 entries before rea

Found 1 tile(s) for observation 078 NIRCam EO-18 GSEG3E2-OTB-20190318 ModA-SUBGRISM128 copy
Found 1 visits with numbers: [1]
APTObservationParams Dictionary holds 78 entries after reading template (+1 entries)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Observation `079` labelled `NIRCam EO-19 GSEG3E2-OTB-20190318 ModA-SUBGRISM64 copy` uses template `NircamEngineeringImaging`
APTObservationParams Dictionary holds 78 entries before reading template
Primary dither element PrimaryDithers not found, use default primary dithers value (1).
Number of dithers: 1 primary * 1 subpixel = 1
Dictionary read from template has 1 entries.
Found 1 tile(s) for observation 079 NIRCam EO-19 GSEG3E2-OTB-20190318 ModA-SUBGRISM64 copy
Found 1 visits with numbers: [1]
APTObservationParams Dictionary holds 79 entries after reading template (+1 entries)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Ob

Found 1 tile(s) for observation 127 NIRCam EO-3 GSEG3DR3-OTB-20200203 ModB-SUB64P
Found 1 visits with numbers: [1]
APTObservationParams Dictionary holds 127 entries after reading template (+1 entries)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Observation `128` labelled `NIRCam EO-4 GSEG3DR3-OTB-20200203 ModA-SUBGRISM256` uses template `NircamEngineeringImaging`
APTObservationParams Dictionary holds 127 entries before reading template
Primary dither element PrimaryDithers not found, use default primary dithers value (1).
Number of dithers: 1 primary * 1 subpixel = 1
Dictionary read from template has 1 entries.
Found 1 tile(s) for observation 128 NIRCam EO-4 GSEG3DR3-OTB-20200203 ModA-SUBGRISM256
Found 1 visits with numbers: [1]
APTObservationParams Dictionary holds 128 entries after reading template (+1 entries)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Observation `129`

Number of dithers: 1 primary * 1 subpixel = 1
Dictionary read from template has 1 entries.
Found 1 tile(s) for observation 170 NIRCam EO-2 GSEG3DR4-EMTB-20200615 ModB-SUB160P
Found 1 visits with numbers: [1]
APTObservationParams Dictionary holds 170 entries after reading template (+1 entries)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Observation `171` labelled `NIRCam EO-3 GSEG3DR4-EMTB-20200615 ModB-SUB64P` uses template `NircamEngineeringImaging`
APTObservationParams Dictionary holds 170 entries before reading template
Primary dither element PrimaryDithers not found, use default primary dithers value (1).
Number of dithers: 1 primary * 1 subpixel = 1
Dictionary read from template has 1 entries.
Found 1 tile(s) for observation 171 NIRCam EO-3 GSEG3DR4-EMTB-20200615 ModB-SUB64P
Found 1 visits with numbers: [1]
APTObservationParams Dictionary holds 171 entries after reading template (+1 entries)
+++++++++++++++++++++++++++++++++

Found 1 tile(s) for observation 201 NIRCam EO-11 GSEG3RR-TEL-20200711 ModA/B-FULL
Found 1 visits with numbers: [1]
APTObservationParams Dictionary holds 201 entries after reading template (+1 entries)
Finished reading APT xml file.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Number of entries before expanding dithers: 201
Number of entries after expanding dithers:  201
Expanded table for Observation 001 has 1 entries
Expanded table for Observation 002 has 1 entries
Expanded table for Observation 003 has 1 entries
Expanded table for Observation 004 has 1 entries
Expanded table for Observation 005 has 1 entries
Expanded table for Observation 006 has 1 entries
Expanded table for Observation 007 has 1 entries
Expanded table for Observation 008 has 1 entries
Expanded table for Observation 009 has 1 entries
Expanded table for Observation 010 has 1 entries
Expanded table for Observation 011 has 1 entries
Expanded table for Observation 0

...

Checking jw00617195001_02102_00001_nrca1_uncal.fits
----------------------------------------------------
Requested readout pattern RAPID is valid. Using the nframe = 1 and nskip = 0
Multiple parent apertures: NRCA1_FULL; NRCA5_GRISM128_F444W
MISMATCH: EXP_TYPE, in exp table: NRC_IMAGE, in file: NRC_TSIMAGE

Checking jw00617195001_02102_00001_nrca1_rate.fits
----------------------------------------------------
Requested readout pattern RAPID is valid. Using the nframe = 1 and nskip = 0
Multiple parent apertures: NRCA1_FULL; NRCA5_GRISM128_F444W
MISMATCH: EXP_TYPE, in exp table: NRC_IMAGE, in file: NRC_TSIMAGE



Checking jw00617195001_02102_00001_nrca3_uncal.fits
----------------------------------------------------
Requested readout pattern RAPID is valid. Using the nframe = 1 and nskip = 0
MISMATCH: EXP_TYPE, in exp table: NRC_IMAGE, in file: NRC_TSIMAGE

Checking jw00617195001_02102_00001_nrca3_rate.fits
----------------------------------------------------
Requested readout pattern RA

Requested readout pattern RAPID is valid. Using the nframe = 1 and nskip = 0
MISMATCH: LONGPUPIL, in exp table: CLEAR, in file: FLAT
MISMATCH: EXP_TYPE, in exp table: NRC_IMAGE, in file: NRC_DARK
MISMATCH: SUBSTRT1, in exp table: 665, in file: 709
MISMATCH: SUBSTRT2, in exp table: 701, in file: 745

Checking jw00617197001_02102_00001_nrcblong_dark.fits
----------------------------------------------------
Requested readout pattern RAPID is valid. Using the nframe = 1 and nskip = 0
MISMATCH: LONGPUPIL, in exp table: CLEAR, in file: FLAT
MISMATCH: EXP_TYPE, in exp table: NRC_IMAGE, in file: NRC_DARK
MISMATCH: SUBSTRT1, in exp table: 665, in file: 709
MISMATCH: SUBSTRT2, in exp table: 701, in file: 745





OBSERVATION: 198

Total number of expected files: 5
Expected detectors used: ['NRCB1', 'NRCB2', 'NRCB3', 'NRCB4', 'NRCBLONG']
Found uncal files:
/Volumes/LaCie/gseg_validation_tests/gseg3_july2020_RunForTheRecord/JWST/jw00617198001_02102_00001_nrcb1/jw00617198001_02102_00001_nrcb1_uncal

Requested readout pattern RAPID is valid. Using the nframe = 1 and nskip = 0
MISMATCH: SHORTPUPIL, in exp table: CLEAR, in file: FLAT
MISMATCH: EXP_TYPE, in exp table: NRC_IMAGE, in file: NRC_DARK

Checking jw00617199001_02102_00001_nrcb3_dark.fits
----------------------------------------------------
Requested readout pattern RAPID is valid. Using the nframe = 1 and nskip = 0
MISMATCH: SHORTPUPIL, in exp table: CLEAR, in file: FLAT
MISMATCH: EXP_TYPE, in exp table: NRC_IMAGE, in file: NRC_DARK



Checking jw00617199001_02102_00001_nrcb4_uncal.fits
----------------------------------------------------
Requested readout pattern RAPID is valid. Using the nframe = 1 and nskip = 0
MISMATCH: SHORTPUPIL, in exp table: CLEAR, in file: FLAT
MISMATCH: EXP_TYPE, in exp table: NRC_IMAGE, in file: NRC_DARK

Checking jw00617199001_02102_00001_nrcb4_dark.fits
----------------------------------------------------
Requested readout pattern RAPID is valid. Using the nframe = 1 and nskip = 0
MISMATCH: SHOR

Requested readout pattern BRIGHT1 is valid. Using the nframe = 1 and nskip = 1
No inconsistencies. File header info correct.

Checking jw00617200001_02101_00001_nrcb3_rate.fits
----------------------------------------------------
Requested readout pattern BRIGHT1 is valid. Using the nframe = 1 and nskip = 1
No inconsistencies. File header info correct.



Checking jw00617200001_02101_00001_nrcb4_uncal.fits
----------------------------------------------------
Requested readout pattern BRIGHT1 is valid. Using the nframe = 1 and nskip = 1
No inconsistencies. File header info correct.

Checking jw00617200001_02101_00001_nrcb4_rate.fits
----------------------------------------------------
Requested readout pattern BRIGHT1 is valid. Using the nframe = 1 and nskip = 1
No inconsistencies. File header info correct.



Checking jw00617200001_02101_00001_nrcblong_uncal.fits
----------------------------------------------------
Requested readout pattern BRIGHT1 is valid. Using the nframe = 1 and ns

Requested readout pattern BRIGHT1 is valid. Using the nframe = 1 and nskip = 1
No inconsistencies. File header info correct.

Checking jw00617201001_02101_00001_nrcb4_rate.fits
----------------------------------------------------
Requested readout pattern BRIGHT1 is valid. Using the nframe = 1 and nskip = 1
No inconsistencies. File header info correct.



Checking jw00617201001_02101_00001_nrcblong_uncal.fits
----------------------------------------------------
Requested readout pattern BRIGHT1 is valid. Using the nframe = 1 and nskip = 1
No inconsistencies. File header info correct.

Checking jw00617201001_02101_00001_nrcblong_rate.fits
----------------------------------------------------
Requested readout pattern BRIGHT1 is valid. Using the nframe = 1 and nskip = 1
No inconsistencies. File header info correct.



